In [49]:
import blackjack_engine
import random
import numpy as np
import pandas as pd

In [86]:
# from blackjack_api_example.py
def padarray(A, size):
    if len(A) == 0:
        A = ['']
    t = size - len(A)
    arr = np.pad(A, pad_width=(0, t), mode='empty')
    
    return arr.astype('<U32')

def player_bot(hand):
    """ makes random choice, or makes decisions using neural net etc"""
    # decision making code goes here
    return random.choice(['hit', 'stay']) 

def play_blackjack_game():
    # initialize a game with shuffled deck and hands dealt to dealer and player:
    game = blackjack_engine.BlackjackGame()
    moves = []
    game_states = []
    game_results = []
    if game.is_finished == False: # if nobody got dealt a blackjack, loop through decisions
        while True: 
            #=====================================================================================================
            #=====================================================================================================
            #=====================================================================================================
            player_hand = game.player_hands[0].cards[:]
            dealer_card = game.dealer_hand.cards[0]
            game_state_row = {'player_hand':player_hand,'dealer_card':dealer_card}
            game_states.append(game_state_row)
            #=====================================================================================================
            #=====================================================================================================
            #=====================================================================================================
            hit_or_stay = player_bot(game.player_hands)
            moves.append(hit_or_stay)
            game.player_move(hit_or_stay)

            if not game.is_finished:
                continue # game is not finished so continue loop
            if game.is_finished:
                [game_results.append(game.result) for game_result in np.arange(len(moves))]
                break # game is finished so exit loop
    return game_states, moves, game_results

# implement a training set generation routine: give the API random actions and see how games turn out
def build_training_set(n_pts):
    '''
    state_of_play is an n x m array of values where n is the number of data points, and m is the max number of features in a hand 
    y is an n x 1 array, where n is the number of data points (number of times the player drew a card, basically) 
    '''
    all_game_states = []
    all_moves = []
    all_game_results = []

    for pt in np.linspace(0,n_pts,n_pts+1):
        [game_states, moves,game_results] = play_blackjack_game()
        all_game_states.append(game_states)
        all_moves.append(moves)
        all_game_results.append(game_results)
    return all_game_states, all_moves, all_game_results

In [87]:
[all_game_states, all_moves, all_game_results] = build_training_set(10)

# save the data with a pickle or something

In [88]:
print(np.shape(all_game_states))
print(np.shape(all_moves))
print(np.shape(all_game_results))

(11,)
(11,)
(11,)


In [90]:
# what does the data look like?
n = 1
print(all_moves)
print(all_game_states)
print(all_game_results)

# it is possible to flatten all_moves, as well as all_game_results, but not all_game_states, because they are incorrect.
# how can we pull the information that we want using the engine? 

[['stay'], ['hit', 'stay'], ['hit'], ['stay'], ['hit'], ['hit', 'hit', 'stay'], ['stay'], ['stay'], ['stay'], ['stay'], ['hit']]
[[{'player_hand': ['8', '9'], 'dealer_card': '10'}], [{'player_hand': ['A', '8'], 'dealer_card': 'J'}, {'player_hand': ['A', '8', '6'], 'dealer_card': 'J'}], [{'player_hand': ['J', '4'], 'dealer_card': '5'}], [{'player_hand': ['J', '10'], 'dealer_card': '10'}], [{'player_hand': ['5', '6'], 'dealer_card': 'J'}], [{'player_hand': ['3', '5'], 'dealer_card': 'Q'}, {'player_hand': ['3', '5', '5'], 'dealer_card': 'Q'}, {'player_hand': ['3', '5', '5', '4'], 'dealer_card': 'Q'}], [{'player_hand': ['6', 'A'], 'dealer_card': 'K'}], [{'player_hand': ['6', '10'], 'dealer_card': 'K'}], [{'player_hand': ['2', '9'], 'dealer_card': 'Q'}], [{'player_hand': ['3', 'J'], 'dealer_card': '3'}], [{'player_hand': ['J', '7'], 'dealer_card': '8'}]]
[[0], [-1, -1], [-1], [1], [1], [1, 1, 1], [-1], [-1], [-1], [1], [-1]]


In [91]:
# need to flatten all of the vectors and train a model

In [92]:
# train the neural network on the previously generated data
def build_model(X,y):
    input_shape = np.shape(X)
    output_shape = np.shape(y) 
    # use a simple sequential dense model to predict which actions will result in a win
    return model

def train_model(untrained_model,training_data):
    training_data = [X_train, y_train, X_test, y_test]
    
    return trained_network

def use_model_to_make_decision(trained_model,state_of_play):

    decision = trained_model.predict(state_of_play)
    while game_is_still_in_play:
        state_of_play = play_blackjack_hand(state_of_play,decision)
        game_is_still_in_play = check_result(state_of_play)
    return state_of_play
    

In [93]:
for row in actions

SyntaxError: invalid syntax (<ipython-input-93-c72098d537c4>, line 1)